<a href="https://colab.research.google.com/github/myDSMLProjects/Deep-Learning/blob/master/raw_model_mpg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

In [0]:
save_path = '.'

In [2]:
tf.__version__

'2.2.0-rc3'

In [0]:
df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

In [4]:
df.head(5)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [0]:
cars = df['name']

In [6]:
cars

0      chevrolet chevelle malibu
1              buick skylark 320
2             plymouth satellite
3                  amc rebel sst
4                    ford torino
                 ...            
393              ford mustang gl
394                    vw pickup
395                dodge rampage
396                  ford ranger
397                   chevy s-10
Name: name, Length: 398, dtype: object

In [0]:
# handling the missing values
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

In [0]:
# pandas to numpy
X = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values

In [9]:
# Building the neural network
model = Sequential()
model.add(Dense(25, input_dim=X.shape[1], activation='relu')) # hidden1
model.add(Dense(10, activation='relu')) # hidden2
model.add(Dense(1)) # Output

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X, y, verbose=2, epochs=100)

Epoch 1/100
13/13 - 0s - loss: 13064.1562
Epoch 2/100
13/13 - 0s - loss: 1030.4706
Epoch 3/100
13/13 - 0s - loss: 941.6262
Epoch 4/100
13/13 - 0s - loss: 174.2743
Epoch 5/100
13/13 - 0s - loss: 141.2920
Epoch 6/100
13/13 - 0s - loss: 113.3960
Epoch 7/100
13/13 - 0s - loss: 100.4741
Epoch 8/100
13/13 - 0s - loss: 95.0682
Epoch 9/100
13/13 - 0s - loss: 97.6284
Epoch 10/100
13/13 - 0s - loss: 92.7464
Epoch 11/100
13/13 - 0s - loss: 93.6262
Epoch 12/100
13/13 - 0s - loss: 92.0554
Epoch 13/100
13/13 - 0s - loss: 95.9909
Epoch 14/100
13/13 - 0s - loss: 94.6854
Epoch 15/100
13/13 - 0s - loss: 92.6526
Epoch 16/100
13/13 - 0s - loss: 91.9128
Epoch 17/100
13/13 - 0s - loss: 93.1650
Epoch 18/100
13/13 - 0s - loss: 91.9377
Epoch 19/100
13/13 - 0s - loss: 93.0336
Epoch 20/100
13/13 - 0s - loss: 95.5114
Epoch 21/100
13/13 - 0s - loss: 91.5287
Epoch 22/100
13/13 - 0s - loss: 88.3162
Epoch 23/100
13/13 - 0s - loss: 90.4517
Epoch 24/100
13/13 - 0s - loss: 92.2624
Epoch 25/100
13/13 - 0s - loss: 90.3456

In [10]:
pred = model.predict(X)
print(f'shape pf prediction : {pred.shape}')
print(pred[:10])

shape pf prediction : (398, 1)
[[12.262991 ]
 [11.07044  ]
 [11.894886 ]
 [14.142155 ]
 [13.54645  ]
 [ 8.718355 ]
 [ 7.1609817]
 [ 8.438959 ]
 [ 8.304792 ]
 [ 8.921631 ]]


In [11]:
print(metrics.r2_score(y, pred))

0.16898958240107165


In [12]:
score = np.sqrt(metrics.mean_squared_error(pred, y))
print(f'RMSE : {score}')

RMSE : 7.116076864254066


In [13]:
# Sample predictions
for i in range(10):
  print(f'{i+1}. Car name: {cars[i]}, Actual MPG: {y[i]}, Predicted MPG: {pred[i]}')

1. Car name: chevrolet chevelle malibu, Actual MPG: 18.0, Predicted MPG: [12.262991]
2. Car name: buick skylark 320, Actual MPG: 15.0, Predicted MPG: [11.07044]
3. Car name: plymouth satellite, Actual MPG: 18.0, Predicted MPG: [11.894886]
4. Car name: amc rebel sst, Actual MPG: 16.0, Predicted MPG: [14.142155]
5. Car name: ford torino, Actual MPG: 17.0, Predicted MPG: [13.54645]
6. Car name: ford galaxie 500, Actual MPG: 15.0, Predicted MPG: [8.718355]
7. Car name: chevrolet impala, Actual MPG: 14.0, Predicted MPG: [7.1609817]
8. Car name: plymouth fury iii, Actual MPG: 14.0, Predicted MPG: [8.438959]
9. Car name: pontiac catalina, Actual MPG: 14.0, Predicted MPG: [8.304792]
10. Car name: amc ambassador dpl, Actual MPG: 15.0, Predicted MPG: [8.921631]


In [0]:
# Save neural network structure to json (no weights)
model_json = model.to_json()
with open(os.path.join(save_path, 'network.json'), 'w') as json_file:
  json_file.write(model_json)

# Save neural network structure to YAML (no weights)
model_yaml = model.to_yaml()
with open(os.path.join(save_path, 'network.yaml'), 'w') as yaml_file:
  yaml_file.write(model_yaml)

# Save neural network structure to HDF5 (saves everything, suggested)
model.save(os.path.join(save_path, 'network.h5'))

In [20]:
from tensorflow.keras.models import load_model
model2 = load_model(os.path.join(save_path, 'network.h5'))
pred = model2.predict(X)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred, y))
print(f'After load score(RMSE: {score}')


After load score(RMSE: 7.116076864254066
